# Kaggle cell

In [1]:
# !pip install wandb timm numpy pytorch-metric-learning faiss-gpu --upgrade
# !rm -r kaggle_happywhale_2022
# !git clone https://github.com/btseytlin/kaggle_happywhale_2022.git
# import sys
# sys.path.append('kaggle_happywhale_2022')
#
# import wandb
#
# try:
#     from kaggle_secrets import UserSecretsClient
#     user_secrets = UserSecretsClient()
#     api_key = user_secrets.get_secret("WANDB")
#     wandb.login(key=api_key)
#     anonymous = None
# except:
#     anonymous = "must"
#     wandb.login(anonymous=anonymous)
#     print('wand secret missing')

# Imports

In [2]:
import timm

import wandb
import pandas as pd
import numpy as np
import plotly.express as px
import torch
import wandb
from matplotlib import pyplot as plt


In [3]:
from happywhale import (ImageDataMoodule,
                        ImageBackbone,
                        MetricLearner,
                        seed_torch,
                        load_train_test_dfs,
                        get_cv_splits)

In [4]:
%matplotlib inline

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
# seed_torch(42)

# Config

In [7]:
OFFLINE = False
EXP_NAME = 'metric_learning'
tags = ['dataset_base_128', 'backbone_efficientnet_b0', 'metric_learning', 'contrastive_loss']
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


TRAIN_IMG_DIR = '../input/jpeg-happywhale-128x128/train_images-128-128/train_images-128-128'
TEST_IMG_DIR = '../input/jpeg-happywhale-128x128/test_images-128-128/test_images-128-128'
TRAIN_CSV_PATH = '../input/happy-whale-and-dolphin/train.csv'
TEST_CSV_PATH = '../input/happy-whale-and-dolphin/sample_submission.csv'




BACKBONE = 'efficientnet_b0'
LR = 1e-3
BACKBONE_EMBEDDING_DIM = 1000
EMBEDDING_SIZE = 256
CLASS_SUBSET = 10

N_EPOCHS = 10
BATCH_SIZE = 64
NUM_WORKERS = 2


TRAINER_KWARGS = dict(
    max_epochs=N_EPOCHS,
    devices="auto",
    accelerator="auto",
    gradient_clip_val=1,
    accumulate_grad_batches=2,
    # stochastic_weight_avg=True,
    # fast_dev_run=True,
)

if DEVICE != 'cpu':
    TRAINER_KWARGS.update(
        dict(
            # amp_backend='apex',
            # amp_level='O2',
            precision=16,
        )
    )

# Local overrides

In [8]:
TRAINER_KWARGS['fast_dev_run'] = 1
TRAIN_IMG_DIR = '../data/images_128/train_images-128-128'
TEST_IMG_DIR = '../data/images_128/test_images-128-128'
TRAIN_CSV_PATH = '../data/train.csv'
TEST_CSV_PATH = '../data/test.csv'
EXP_NAME = 'LOCAL_TEST'
OFFLINE = True

# CV: split and prepare datamodules

In [9]:
train_df, test_df = load_train_test_dfs(
    train_csv_path=TRAIN_CSV_PATH,
    test_csv_path=TEST_CSV_PATH,
    train_images_path=TRAIN_IMG_DIR,
    test_images_path=TEST_IMG_DIR,
)
print(train_df.shape, test_df.shape)

(51033, 5) (27956, 4)


In [10]:
if CLASS_SUBSET:
    classes = train_df.individual_id.value_counts()
    train_df = train_df[train_df.individual_id.isin(classes.head(CLASS_SUBSET).index)]
    print(train_df.shape)

(1750, 5)


In [11]:
cv_splits = get_cv_splits(train_df)

In [12]:
len(cv_splits[0].train), len(cv_splits[0].val)

(1260, 140, 350)

In [13]:
split_datamodules = []
for split in cv_splits:
    split_train_df = train_df.iloc[split.train]
    split_val_df = train_df.iloc[split.val]
    split_test_df = train_df.iloc[split.test]
    datamodule = ImageDataMoodule(
        train_df=split_train_df,
        val_df=split_val_df,
        test_df=split_test_df,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        sampler='m_per_class',
    )
    datamodule.setup()
    split_datamodules.append(datamodule)

/Users/btseytlin/.pyenv/versions/3.8.12/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:121: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/Users/btseytlin/.pyenv/versions/3.8.12/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:159: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/Users/btseytlin/.pyenv/versions/3.8.12/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:114: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/Users/btseytlin/.pyenv/versions/3.8.12/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:152: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_

# Train model

In [ ]:
models = []
for i, datamodule in enumerate(split_datamodules):
    backbone = ImageBackbone(model_name=BACKBONE)
    model = MetricLearner(
        backbone=backbone,
        lr=LR,
        num_labels=len(datamodule.label_encoder.classes_),
        num_training_steps=len(datamodule.train)//datamodule.batch_size * N_EPOCHS,
        trainer_kwargs=TRAINER_KWARGS,
        backbone_embedding_dim=BACKBONE_EMBEDDING_DIM,
        embedding_size=EMBEDDING_SIZE,
        offline=OFFLINE,
    )

    run = wandb.init(
        project='kaggle_happywhale',
        name=EXP_NAME + f'_fold_{i}',
        tags=tags,
    )

    model.fit(datamodule)
    models.append(model.cpu())
    run.finish()
    break

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
wandb: Currently logged in as: btseytlin (use `wandb login --relogin` to force relogin)


/Users/btseytlin/.pyenv/versions/3.8.12/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name     | Type               | Params
------------------------------------------------
0 | backbone | ImageBackbone      | 5.3 M 
1 | mlp      | Sequential         | 322 K 
2 | loss     | ContrastiveLoss    | 0     
3 | miner    | BatchEasyHardMiner | 0     
------------------------------------------------
5.6 M     Trainable params
0         Non-trainable params
5.6 M     Total params
22.442    Total estimated model params size (MB)
/Users/btseytlin/.pyenv/versions/3.8.12/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of th

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]


If you're looking for MAP@10 instead of MAP@R, then you should use 'mean_average_precision' rather than mean_average_precision_at_r
